In [ ]:
import numpy as np
import os
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report
from tqdm import tqdm

In [ ]:
# Load the pre-trained model
model = load_model('<path_to_model>/s3FN.h5')

# Load PCA components and mean
pca_components = np.load(r'<path_to_pca>/Blueberry_train_cubes_PCA_components.npy')
pca_mean = np.load(r'<path_to_pca>/Blueberry_train_cubes_PCA_mean.npy')

# Set input directory for testing data
input_directory = r'<path_to_input_data>'

In [ ]:
model.summary()

In [ ]:
# Collect all true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate over each class directory
for class_label in tqdm(os.listdir(input_directory), desc='Processing classes'):
    
    class_dir = os.path.join(input_directory, class_label)
    
    if not os.path.isdir(class_dir):
        continue
    
    # Process each image in the class directory
    for file in tqdm(os.listdir(class_dir), desc=f'Processing {class_label}'):
        
        if not file.endswith('.npy'):
            continue
        
        file_path = os.path.join(class_dir, file)
        image = np.load(file_path)  # Shape: (250, 150, 252)
        
        # Generate mask to exclude background (assuming sum of bands > 0)
        sum_bands = np.sum(image, axis=2)
        mask = sum_bands > 0  # Adjust threshold if necessary
        
        height, width, _ = image.shape
        cube_size = 32
        max_h = height - cube_size
        max_w = width - cube_size
        
        cubes = []
        attempts = 0
        max_attempts = 1000
        
        # Collect 10 valid cubes
        while len(cubes) < 20 and attempts < max_attempts:
            
            h_start = np.random.randint(0, max_h + 1)
            w_start = np.random.randint(0, max_w + 1)
            
            # Check if the entire cube is within non-masked area
            if mask[h_start:h_start+cube_size, w_start:w_start+cube_size].all():
                
                cube = image[h_start:h_start+cube_size, w_start:w_start+cube_size, :]
                cubes.append(cube)
                
            attempts += 1
        
        if len(cubes) < 10:
            
            print(f"Warning: Only found {len(cubes)} cubes for {file}. Skipping.")
            
            continue
        
        # Process each cube and predict
        pred_classes = []
        
        for cube in cubes:
            
            # Apply PCA
            cube_flat = cube.reshape(-1, 224)  
            cube_centered = cube_flat - pca_mean
            cube_pca = np.dot(cube_centered, pca_components.T) 
            cube_pca = cube_pca.reshape(32, 32, 142)
            
            # Z-score normalization per band
            mean = np.mean(cube_pca, axis=(0, 1))
            std = np.std(cube_pca, axis=(0, 1))
            std[std == 0] = 1e-6  # Avoid division by zero
            cube_normalized = (cube_pca - mean) / std
            
            # Prepare input for the model (add channel and batch dimensions)
            cube_input = np.expand_dims(cube_normalized, axis=-1)  # (32, 32, 23, 1)
            cube_input = np.expand_dims(cube_input, axis=0)        # (1, 32, 32, 23, 1)
            
            # Predict
            pred = model.predict(cube_input, verbose=0)
            pred_class = np.argmax(pred, axis=1)[0]
            pred_classes.append(pred_class)
        
        # Majority voting
        majority_vote = np.bincount(pred_classes).argmax()
        predicted_labels.append(majority_vote)
        true_labels.append(class_label)

In [ ]:
# Generate classification report
CLASS_NAMES = ['good', 'bad']

# Then use this for mapping instead of sorted(unique_labels)
label_to_index = {label: idx for idx, label in enumerate(CLASS_NAMES)}

#unique_labels = sorted(np.unique(true_labels))
#label_to_index = {label: idx for idx, label in enumerate(unique_labels)}

# Convert string labels to indices using predefined CLASS_NAMES
true_indices = [label_to_index[label] for label in true_labels]

# Get all possible class indices
all_labels = list(range(len(CLASS_NAMES)))

print(classification_report(
    true_indices, 
    predicted_labels, 
    labels=all_labels,
    target_names=CLASS_NAMES,
    zero_division=0,
    digits=4
))